In [1]:
import torch
from torch import nn
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets.mpr_dataset import MPR_Dataset,MPR_Dataset_H5
from tqdm import tqdm_notebook as tqdm
import yaml
import pandas as pd
import numpy as np
from models import ShuffleNetv2
from sklearn.metrics import accuracy_score, f1_score
from ast import literal_eval
# from visualize_results import label_predictions_to_images
import os
import cv2
import copy
import matplotlib.pyplot as plt

In [2]:
with open('./config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config['device'] = 'cpu'
root_dir = config["data"]["root_dir"]
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = MPR_Dataset(root_dir, partition="train", config=config["data"], transform=transform)

test_loader = DataLoader(
    MPR_Dataset(root_dir, partition="test", config=config["data"], transform=transform), shuffle=False,
    batch_size=32)
test_loader.dataset.df['LABEL'] = test_loader.dataset.labels

device = torch.device("cuda")
model = ShuffleNetv2(n_classes=3)
# PATH_WEIGHTS = '/home/petryshak/CoronaryArteryPlaqueIdentification/experiments/exp2/models/'
PATH_WEIGHTS = 'model_model_34_val_f1=0.9360136.pth'
# model.load_state_dict(torch.load(PATH_WEIGHTS, map_location={'cuda:0': 'cpu'}))
model.load_state_dict(torch.load(PATH_WEIGHTS,map_location={'cuda:0': 'cpu'}))
model.model.fc = nn.Sequential()
model.model.conv5 = model.model.conv5[:-2]
model.to(device)
model.eval()

ShuffleNetv2(
  (model): ShuffleNetV2(
    (conv1): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (stage2): Sequential(
      (0): InvertedResidual(
        (branch1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU(inplace=True)
        )
        (branch2): Sequential(
          (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNo

In [3]:
predictions = [] 

with torch.no_grad():
    
    for step, (x, y) in enumerate(tqdm(test_loader)):
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        _, predicted = torch.max(output, 1)
        predictions.extend(output.cpu().detach().numpy())

In [4]:
p_test_df = test_loader.dataset.df
p_test_df = p_test_df[p_test_df['MPR_VIEWPOINT_INDEX']%1 == 0].reset_index()
p_test_df['PRED'] = pd.Series(predictions)
p_test_df['PATIENT'] = p_test_df['IMG_PATH'].apply(lambda s: s.split('/')[1])

In [ ]:
p_test_df.to_csv('xgboost_train_features.csv')

In [5]:
p_test_df.to_pickle('xgboost_test_features.csv')
